In [ ]:
import pandas as pd
import re
import os
from glob import glob

# Initializing variables
data = []

# Getting the list of .txt files in the folder. In this case, I created a folder called E2E in the C drive.
txt_files = glob('C:/E2E/*.txt')

# Loop to process each file
for file in txt_files:
    # Getting the file name (it has the NE name). 
    file_name = os.path.basename(file)
    file_name = os.path.splitext(file_name)[0]
    
    # Opening the text file
    with open(file, 'r') as f:
        current_mo = None
        for line in f:
            # Searching for MO
            mo_match = re.match(r'/\* MOC = (.*) \*/', line.strip())
            if mo_match:
                current_mo = mo_match.group(1)
            # Searching for parameters and their values, considering the required info are in "SET" and "MOD" rows. 
            param_match = re.match(r'(SET|MOD) (.*):(.*)', line.strip())
            if param_match and current_mo:
                params = param_match.group(3).split(',')
                for param in params:
                    param_parts = param.split('=')
                    if len(param_parts) == 2:
                        param_name, param_value = param_parts
                        param_value = param_value.strip().rstrip(';')
                        command_match = re.search(r'(SET|MOD)\s([^:]+):', line.strip())
                        if command_match:
                            command = command_match.group(2)
                        else:
                            command = ""
                        data.append([current_mo.strip(), command.strip(), param_name.strip(), param_value, file_name])

# Creating the DataFrame
df = pd.DataFrame(data, columns=["MO", "Command", "Parameter", "Parameter_Value", "Element"])

df.drop_duplicates(subset=["MO", "Command", "Parameter", "Parameter_Value", "Element"], inplace=True)

# Adding the "Value_Count" column
df["Value_Count"] = df.groupby(["Command", "Parameter", "Parameter_Value"])["Element"].transform("count")

# Adding the "Element_Count" column
df["Element_Count"] = df.groupby(["Parameter", "Command"])["Element"].transform("count")

# Printing the DataFrame
print(df)